In [1]:
from dotenv import load_dotenv
import asyncio
import json
import os
import requests
from googleapiclient.discovery import build
import pprint
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models import ChatOpenAI
from autogen import config_list_from_json
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen.agentchat.agent import Agent
from autogen import UserProxyAgent
from openai import OpenAI
import autogen
import time

from collections import defaultdict
from queue import Queue

from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.tools import Tool, StructuredTool
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.memory import ConversationBufferMemory
from langchain.agents import Agent
from typing import List, Literal
from langchain.schema.messages import (
    AIMessage,
    AnyMessage,
    BaseMessage,
    ChatMessage,
    HumanMessage,
    SystemMessage,
    get_buffer_string,
)
from InboxAgent import InboxAgent
from UserProxyAgent import UserProxyAgent
from AgentNetwork import AgentNetwork


from InteractionVisualizer import InteractionVisualizer
from NetworkServer import NetworkServer
from flask import Flask

from threading import Thread
from NetworkRoutes import create_network_blueprint


from langchain_core.pydantic_v1 import BaseModel, Field


In [2]:
load_dotenv()  # take environment variables from .env.

google_search_api_key = os.getenv('GOOGLE_SEARCH_API_KEY')
google_search_cse_id = os.getenv('GOOGLE_SEARCH_CSE_ID')
browserless_api_key = os.getenv('BROWSERLESS_API_KEY')
config_list_gpt35 = config_list_from_json("OAI_CONFIG_LIST_35")
config_list_gpt4 = config_list_from_json("OAI_CONFIG_LIST_4")

client = OpenAI()

In [3]:
agent_network = AgentNetwork(available_agent_instantiations=[InboxAgent, UserProxyAgent])

# class GetConnectionsByNameUUID(BaseModel):
#     agent_name_uuid: str = Field(
#         description="The name_uuid of the agent you want to get connections for. If you want your own connections, use your own name_uuid."
#     )

# class SendMessageByNameUUID(BaseModel):
#     message: str = Field(
#         description="The message you want to send to the recipient"
#     )
#     agent_name_uuid_sender: str = Field(
#         description="The name_uuid of the agent sending the message. It's usually your own name_uuid."
#     )
#     agent_name_uuid_recipient: str = Field(
#         description="The name_uuid of the agent receiving the message"
#     )

class GetWordLength(BaseModel):
    word: str = Field(
        description="The word you want to get the length of"
    )


In [4]:
# Step 1: Initialize the language model
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

# Step 2: Define the tools (functions) that the assistant can use
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# def send_message_by_name_uuid(message: str, agent_name_uuid_sender: str, agent_name_uuid_recipient: str):
#     """Send a message to a specific agent."""
#     agent_network.send_message_by_name_uuid(message, agent_name_uuid_sender, agent_name_uuid_recipient)

# # adding additional tools
# get_connections_by_name_uuid = StructuredTool.from_function(
#     agent_network.get_connections_by_name_uuid,
#     name="get_connections_by_name_uuid",
#     description="Look at your current connections to see who you can talk to/send messages to. You should input YOUR OWN name_uuid to see who you can talk to.",
#     args_schema=GetConnectionsByNameUUID,
#     return_direct=True,
# )

# send_message_by_name_uuid = StructuredTool.from_function(
#     send_message_by_name_uuid,
#     name="send_message_by_name_uuid",
#     description="Send a message to a specific agent.",
#     args_schema=SendMessageByNameUUID,
#     return_direct=True,
# )


tools = [
    StructuredTool.from_function(
        get_word_length,
        name="get_word_length",
        description="Returns the length of a word.",
        args_schema=GetWordLength,
    ),
    # get_connections_by_name_uuid,
    # send_message_by_name_uuid,
]

In [5]:
agent1 = InboxAgent(
    name="jokebot_teller",
    llm=llm,
    tools=tools,
    description="you are a helpful assistant that likes to tell jokes with other AI agents by sending messages.",
    oai=True,
)

agent2 = InboxAgent(
    name="jokebot_responder",
    llm=llm,
    tools=tools,
    description="you are a helpful assistant that likes to laugh at jokes with other AI agents. you like to send jokes back via messages too!",
    oai=True,
)

agent3 = InboxAgent(
    name="jokebot_responder",
    llm=llm,
    tools=tools,
    description="you are a helpful assistant that likes to laugh at jokes with other AI agents. you like to send jokes back via messages too!",
    oai=True,
)

user1 = UserProxyAgent(
)

agent_network.add_agent(agent1)
agent_network.add_agent(agent2)
agent_network.add_agent(agent3)
agent_network.add_agent(user1)

agent_network.connect_agents(agent1, agent2)
agent_network.connect_agents(user1, agent1)
agent_network.connect_agents(agent1, agent3)


In [ ]:
# network_server = NetworkServer(agent_network)
# network_server.run(use_reloader=False)

loop = asyncio.get_event_loop()
network_server = NetworkServer(agent_network, event_loop=loop)
flask_thread = Thread(target=network_server.run, kwargs={'use_reloader': False, 'allow_unsafe_werkzeug': True})
flask_thread.start()


Werkzeug appears to be used in a production deployment. Consider switching to a production web server instead.


 * Serving Flask app 'AgentNetworkSocket'


 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Jan/2024 15:26:10] "GET /socket.io/?EIO=4&transport=polling&t=Opr3ST_ HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 15:26:10] "POST /socket.io/?EIO=4&transport=polling&t=Opr3SUh&sid=ERwJ23_CJ5VYniY5AAAA HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 15:26:10] "GET /socket.io/?EIO=4&transport=polling&t=Opr3SUj&sid=ERwJ23_CJ5VYniY5AAAA HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_0fe1ae35-c65c-4446-8d45-77915d603a21', 'label': 'jokebot_teller_0fe1ae35-c65c-4446-8d45-77915d603a21', 'type': 'jokebot_teller', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'label': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'label': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'user_093c217e-202e-4635-af9b-c2c7c7b5ec4a', 'label': 'user_093c217e-202e-4635-af9b-c2c7c7b5ec4a', 'type': 'user', 'running': True, 'status': 'Running'}], 'links': [{'id': 'jokebot_teller_0fe1ae35-c65c-4446-8d45-77915d603a21_jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'source': 'jokebot_teller_0fe1ae35-c65c-4446-8d45-77915d603a21', 'target': 'jokebot_respo

...

Client connected
{'nodes': [{'id': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'label': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'label': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'user_093c217e-202e-4635-af9b-c2c7c7b5ec4a', 'label': 'user_093c217e-202e-4635-af9b-c2c7c7b5ec4a', 'type': 'user', 'running': True, 'status': 'Running'}], 'links': [{'id': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac_jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'source': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'target': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'curvature': 0.2, 'rotation': 3.141592653589793}, {'id': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd_user_093c217e-202e-4635-af9b-

127.0.0.1 - - [10/Jan/2024 22:00:29] "GET /socket.io/?EIO=4&transport=websocket&sid=TkZpEgUHjCdL0C8lAAAl HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [10/Jan/2024 22:00:29] "GET /socket.io/?EIO=4&transport=polling&t=OpsTiiR HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 22:00:30] "POST /socket.io/?EIO=4&transport=polling&t=OpsTinZ&sid=eENoP2tfGzD3B8vkAAAn HTTP/1.1" 200 -
127.0.0.1 - - [10/Jan/2024 22:00:30] "GET /socket.io/?EIO=4&transport=polling&t=OpsTina&sid=eENoP2tfGzD3B8vkAAAn HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'label': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'label': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'type': 'jokebot_responder', 'running': False, 'status': 'Stopped'}, {'id': 'user_093c217e-202e-4635-af9b-c2c7c7b5ec4a', 'label': 'user_093c217e-202e-4635-af9b-c2c7c7b5ec4a', 'type': 'user', 'running': True, 'status': 'Running'}], 'links': [{'id': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac_jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'source': 'jokebot_responder_5344c93e-b2b0-4802-ae17-dc89e72722ac', 'target': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd', 'curvature': 0.2, 'rotation': 3.141592653589793}, {'id': 'jokebot_responder_871c6da6-09e6-424c-9441-115b568af3bd_user_093c217e-202e-4635-af9b-

In [17]:
# loop = asyncio.get_event_loop()
# loop.run_until_complete(agent.run())

# agent1_run = asyncio.create_task(agent1.run())
# agent2_run = asyncio.create_task(agent2.run())
# agent3_run = asyncio.create_task(agent3.run())

agent1.start()
agent2.start()
agent3.start()

In [ ]:
agent1.stop()
agent2.stop()
agent3.stop()

127.0.0.1 - - [09/Jan/2024 15:49:38] "GET /socket.io/?EIO=4&transport=websocket&sid=I5NoeTI7qslkDtm2AAAD HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [09/Jan/2024 15:49:39] "GET /socket.io/?EIO=4&transport=polling&t=Opl_Eby HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:49:39] "POST /socket.io/?EIO=4&transport=polling&t=Opl_Eh2&sid=qLi9Zr06UouvBposAAAF HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:49:39] "GET /socket.io/?EIO=4&transport=polling&t=Opl_Eh3&sid=qLi9Zr06UouvBposAAAF HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'label': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'type': 'jokebot_teller', 'status': False}, {'id': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'label': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'type': 'jokebot_responder', 'status': False}, {'id': 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf', 'label': 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf', 'type': 'jokebot_responder', 'status': False}, {'id': 'user_da6ae2ea-ced2-4008-975b-93fa4a386269', 'label': 'user_da6ae2ea-ced2-4008-975b-93fa4a386269', 'type': 'user', 'status': True}], 'links': [{'id': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360_jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'source': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'target': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'curvature': 0.2, 'rotation': 3.141592653589

127.0.0.1 - - [09/Jan/2024 15:50:06] "GET /socket.io/?EIO=4&transport=websocket&sid=qLi9Zr06UouvBposAAAF HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:50:06] "GET /socket.io/?EIO=4&transport=polling&t=Opl_LW3 HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [09/Jan/2024 15:50:07] "POST /socket.io/?EIO=4&transport=polling&t=Opl_LWV&sid=MaVVubId23w0bpzqAAAH HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:50:07] "GET /socket.io/?EIO=4&transport=polling&t=Opl_LWY&sid=MaVVubId23w0bpzqAAAH HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'label': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'type': 'jokebot_teller', 'status': False}, {'id': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'label': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'type': 'jokebot_responder', 'status': False}, {'id': 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf', 'label': 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf', 'type': 'jokebot_responder', 'status': False}, {'id': 'user_da6ae2ea-ced2-4008-975b-93fa4a386269', 'label': 'user_da6ae2ea-ced2-4008-975b-93fa4a386269', 'type': 'user', 'status': True}], 'links': [{'id': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360_jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'source': 'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360', 'target': 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594', 'curvature': 0.2, 'rotation': 3.141592653589

In [9]:
agent1.add_error("test error1")

{'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360': <InboxAgent.InboxAgent object at 0x000001E29B65ABB0>, 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594': <InboxAgent.InboxAgent object at 0x000001E29B65AB50>, 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf': <InboxAgent.InboxAgent object at 0x000001E29B65ABE0>, 'user_da6ae2ea-ced2-4008-975b-93fa4a386269': <UserProxyAgent.UserProxyAgent object at 0x000001E29B65AC40>}
{'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360': <InboxAgent.InboxAgent object at 0x000001E29B65ABB0>, 'jokebot_responder_10969fc8-e693-4384-a685-fe1711734594': <InboxAgent.InboxAgent object at 0x000001E29B65AB50>, 'jokebot_responder_9e763d18-baa9-4309-91e5-c32bca6d82bf': <InboxAgent.InboxAgent object at 0x000001E29B65ABE0>, 'user_da6ae2ea-ced2-4008-975b-93fa4a386269': <UserProxyAgent.UserProxyAgent object at 0x000001E29B65AC40>}
{'jokebot_teller_7c7aab7f-d4c1-46df-b318-1d64459b5360': <InboxAgent.InboxAgent object at 0x000001E29B65ABB0>, 'jokebot

In [7]:
agent2.add_message("tell all of your connections a joke!", user1.get_description_json()['name_uuid'])
agent3.add_message("tell all of your connections a joke!", user1.get_description_json()['name_uuid'])


# user1.send_message("tell all of your connections (including me) a joke! tell a max of 2 jokes to each person", agent1.get_description_json()['name_uuid'])

In [ ]:
for i in range(5):
    user1.send_message("test", agent1.get_description_json()['name_uuid'])
    time.sleep(3)

127.0.0.1 - - [09/Jan/2024 13:19:00] "GET /socket.io/?EIO=4&transport=websocket&sid=5v-qN2K43UuYZzTHAAAu HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [09/Jan/2024 13:19:00] "GET /socket.io/?EIO=4&transport=polling&t=OplSlud HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 13:19:00] "POST /socket.io/?EIO=4&transport=polling&t=OplSlzq&sid=051MvmG8XGNkAVzwAAAw HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 13:19:00] "GET /socket.io/?EIO=4&transport=polling&t=OplSlzs&sid=051MvmG8XGNkAVzwAAAw HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'label': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'type': 'jokebot_teller', 'status': False}, {'id': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'label': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'type': 'jokebot_responder', 'status': False}, {'id': 'jokebot_responder_08a8d7d9-5bae-4a66-9537-1e26e685fbe5', 'label': 'jokebot_responder_08a8d7d9-5bae-4a66-9537-1e26e685fbe5', 'type': 'jokebot_responder', 'status': False}, {'id': 'user_8d173ce0-2d6f-46d1-9e86-bd11c20b4128', 'label': 'user_8d173ce0-2d6f-46d1-9e86-bd11c20b4128', 'type': 'user', 'status': True}], 'links': [{'id': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4_jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'source': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'target': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'curvature': 0.2, 'rotation': 3.141592653589

127.0.0.1 - - [09/Jan/2024 13:19:05] "GET /socket.io/?EIO=4&transport=websocket&sid=051MvmG8XGNkAVzwAAAw HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 13:19:05] "GET /socket.io/?EIO=4&transport=polling&t=OplSnKq HTTP/1.1" 200 -


...

Client connected
{'nodes': [{'id': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'label': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'type': 'jokebot_teller', 'status': False}, {'id': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'label': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'type': 'jokebot_responder', 'status': False}, {'id': 'jokebot_responder_08a8d7d9-5bae-4a66-9537-1e26e685fbe5', 'label': 'jokebot_responder_08a8d7d9-5bae-4a66-9537-1e26e685fbe5', 'type': 'jokebot_responder', 'status': False}, {'id': 'user_8d173ce0-2d6f-46d1-9e86-bd11c20b4128', 'label': 'user_8d173ce0-2d6f-46d1-9e86-bd11c20b4128', 'type': 'user', 'status': True}], 'links': [{'id': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4_jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'source': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'target': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'curvature': 0.2, 'rotation': 3.141592653589

127.0.0.1 - - [09/Jan/2024 15:18:27] "GET /socket.io/?EIO=4&transport=websocket&sid=nzgky07ZgJc4uNU9AAA- HTTP/1.1" 200 -


Client disconnected


127.0.0.1 - - [09/Jan/2024 15:18:28] "GET /socket.io/?EIO=4&transport=polling&t=Oplu5sB HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:18:28] "POST /socket.io/?EIO=4&transport=polling&t=Oplu5xF&sid=YeYi3p0jbb7zA5JSAABA HTTP/1.1" 200 -
127.0.0.1 - - [09/Jan/2024 15:18:28] "GET /socket.io/?EIO=4&transport=polling&t=Oplu5xI&sid=YeYi3p0jbb7zA5JSAABA HTTP/1.1" 200 -


Client connected
{'nodes': [{'id': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'label': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'type': 'jokebot_teller', 'status': False}, {'id': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'label': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'type': 'jokebot_responder', 'status': False}, {'id': 'jokebot_responder_08a8d7d9-5bae-4a66-9537-1e26e685fbe5', 'label': 'jokebot_responder_08a8d7d9-5bae-4a66-9537-1e26e685fbe5', 'type': 'jokebot_responder', 'status': False}, {'id': 'user_8d173ce0-2d6f-46d1-9e86-bd11c20b4128', 'label': 'user_8d173ce0-2d6f-46d1-9e86-bd11c20b4128', 'type': 'user', 'status': True}], 'links': [{'id': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4_jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'source': 'jokebot_teller_70a5b109-e3d5-41d4-a6b6-543cbe0dfed4', 'target': 'jokebot_responder_a64bb4a5-1531-493e-87fd-110896e12219', 'curvature': 0.2, 'rotation': 3.141592653589

In [ ]:
# setup and deploy the network server

app = Flask(__name__)
network_bp = create_network_blueprint(agent_network)
app.register_blueprint(network_bp, url_prefix='/network')

def run_app():
    app.run(port=5000, use_reloader=False)

flask_thread = Thread(target=run_app)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [04/Jan/2024 20:03:32] "GET /network/data HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2024 20:03:32] "GET /network/data HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2024 23:08:46] "GET /network/data HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2024 23:08:46] "GET /network/data HTTP/1.1" 200 -
